<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/lora-bert-tiny/source/lora-bert-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
if not os.path.exists('/content/drive/MyDrive/lora-bert-tiny/'):
    os.makedirs('/content/drive/MyDrive/lora-bert-tiny/')
output_dir = '/content/drive/MyDrive/lora-bert-tiny/'

In [ ]:
!pip install datasets
!pip install evaluate
!pip install codecarbon
!pip freeze > /content/drive/MyDrive/lora-bert-tiny/requirements.txt

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from codecarbon import EmissionsTracker

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Followed examples from this [Hugging Face NLP course](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api) on how to use the trainer API for fine-tuning.

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
import evaluate

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"])

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return {'total_parameters': parameters, 'trainable_parameters': trainable_parameters}

In [ ]:
import csv

def write_results_to_csv(file_name, results):
  with open(file_name, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))
      writer.writeheader()
      for result in results:
        writer.writerow(result)

Number of times to run each section.

In [ ]:
num_runs = 5

# Full Fine-Tuning



In [ ]:
def fine_tune(epochs, batch_size, learning_rate):
  # Initialize result
  result = {"batch_size": size, "learning_rate": rate}

  # Track emissions
  tracker = EmissionsTracker('fine-tuning-bert-tiny', save_to_file=False)
  try:
    tracker.start()

    # Create the model
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

    # Setup the trainer
    training_args = TrainingArguments(
        "fine-tuning-trainer",
        eval_strategy="epoch",
        per_device_eval_batch_size=size,
        per_device_train_batch_size=size,
        num_train_epochs=epochs,
        learning_rate=rate,
        disable_tqdm=True,
        report_to="none"
    )
    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    result |= trainer.train().metrics

    # Stop tracking emissions
    tracker.stop()

    # Store metrics and delete the tracker
    result |= trainer.evaluate() | get_trainable_parameters(model) | tracker.final_emissions_data.values
  finally:
    del tracker

  return result

Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [ ]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, set_seed
import time

ft_results = []
progress = 0
iterations = len(batch_sizes)*len(learning_rates)*num_runs

# Hyperparameter tuning
start_time = time.time()
for run in range(1, num_runs+1):
  set_seed(run)

  for size in batch_sizes:
    for rate in learning_rates:
      # Track progress
      progress += 1
      print(f"Progress: {progress}/{iterations}")

      # Fine-tune the model and store the results
      ft_result = {"run":run}
      ft_result |= fine_tune(epochs, size, rate)
      ft_results.append(ft_result)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_minutes = runtime_seconds / 60

# Output the best result
max_ft_result = ft_results[0]
for _, result in enumerate(ft_results):
  if result['eval_f1'] > max_ft_result['eval_f1']:
    max_ft_result = result
print(f'\n================ \
        \nTotal Runtime: {runtime_minutes} minutes \
        \nBest Result: \
        \n\tF1={max_ft_result["eval_f1"]} \
        \n\tBatch Size={max_ft_result["batch_size"]} \
        \n\tLearning Rate={max_ft_result["learning_rate"]}'
)

# Save all the results
write_results_to_csv(output_dir + 'ft-results.csv', ft_results)

# Low-Rank Adaptation (LoRA)

In [ ]:
from transformers import AutoModelForSequenceClassification

lora_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

In [ ]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [ ]:
def configure_lora_model(model, rank, alpha):
  # Replace the query and value linear layers with LoRA layers
  for _, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  # Freeze the pre-trained weights
  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False

In [ ]:
def fine_tune_with_lora_model(model, rank, alpha, epochs, batch_size, learning_rate):
  # Initialize result
  result = {"rank": rank, "alpha": alpha, "batch_size": batch_size, "learning_rate": learning_rate}

  # Track emissions
  tracker = EmissionsTracker('lora-bert-tiny', save_to_file=False)
  try:
    tracker.start()

    # Configure an existing model with new LoRA layers
    configure_lora_model(model, rank, alpha)

    # Setup the trainer
    training_args = TrainingArguments(
        "lora-trainer",
        eval_strategy="epoch",
        per_device_eval_batch_size=size,
        per_device_train_batch_size=size,
        num_train_epochs=epochs,
        learning_rate=rate,
        disable_tqdm=True,
        report_to="none"
    )
    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    result |= trainer.train().metrics

    # Stop tracking emissions
    tracker.stop()

    # Store metrics and delete the tracker
    result |= trainer.evaluate() | get_trainable_parameters(model) | tracker.final_emissions_data.values
  finally:
    del tracker

  return result

In [ ]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

In [ ]:
from transformers import Trainer, TrainingArguments, set_seed
import time

lora_results = []
progress = 0
iterations = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)*num_runs

# Hyperparameter tuning
start_time = time.time()
for run in range(1, num_runs+1):
  set_seed(run)

  for rank in ranks:
    for alpha in alphas:
      for size in batch_sizes:
        for rate in learning_rates:
          # Track progress
          progress += 1
          print(f"Progress: {progress}/{iterations}")

          # Fine-tune the LoRA model and store the results
          lora_result = {"run": run}
          lora_result |= fine_tune_with_lora_model(lora_model, rank, alpha, epochs, size, rate)
          lora_results.append(lora_result)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_hours = runtime_seconds / 3600

# Output the best result
max_lora_result = lora_results[0]
for _, result in enumerate(lora_results):
  if result['eval_f1'] > max_lora_result['eval_f1']:
    max_lora_result = result
print(f'\n================ \
        \nTotal Runtime: {runtime_hours} hours \
        \nBest Result: \
        \n\tF1={max_lora_result["eval_f1"]} \
        \n\tRank={max_lora_result["rank"]} \
        \n\tAlpha={max_lora_result["alpha"]} \
        \n\tBatch Size={max_lora_result["batch_size"]} \
        \n\tLearning Rate={max_lora_result["learning_rate"]}'
)

# Save all the results
write_results_to_csv(output_dir + 'lora-results.csv', lora_results)